## 1. Problem Definition

> The goal of this project is to build an AI which will be able to recognize written digits accurately. 

## 2. Data

>The data is taken from Kaggle's Digit Recognzier competition. It contains a dataset of images of  hand written numbers. The training data set, (train.csv), has 785 columns. The first column, called "label", is the digit that was drawn by the user. The rest of the columns contain the pixel-values of the associated image.

>For more information about the competiton check out the page for it [here](https://www.kaggle.com/competitions/digit-recognizer/data)

## 3. Evaluation
>The evaluation metric for this contest is the categorization accuracy, or the proportion of test images that are correctly classified. For example, a categorization accuracy of 0.97 indicates that the AI classified all but 3% of the images.

## Imports

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import display

import keras
from keras import layers
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
gen = image.ImageDataGenerator()

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

import tensorflow_hub as hub

## Data Exploration

In [2]:
train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv", index_col=0)
test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv", index_col=0)

>First lets take a look at some of the sample images from the data set. Initially they are given as an array of integers so they will be converted back into grayscale images for us to view.

In [3]:
def get_image(df, row_index, width=28, height=28):
    '''
    This helper function allows the user to look at the images from the dataset look like.
    Originally they are a data frame of numbers representing each pixel but this function creates
    an image from these numbers which is shown to the user.
    
    df: Data frame
    row_index: Row of dataframe
    width: width of output image
    height: height of output image
    
    returns: None
    '''
    
    # Select the row based on the index
    selected_row = df.iloc[row_index]

    # Convert the selected row to a numpy array
    pixel_values = selected_row.values.astype('uint8')

    # Reshape the pixel values into an image
    reshaped_array = pixel_values.reshape((height, width))

    # Create an image from the array using Pillow
    image = Image.fromarray(reshaped_array)
    
    display(image)


In [4]:
for i in range(10):
    get_image(train, i)

>Now lets get a good idea of the properties of the data frames

In [5]:
train.shape, test.shape

((42000, 784), (28000, 783))

In [6]:
train.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
label,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# Check if "label" is the index of the DataFrame
is_label_index = 'label' in train.index.names

if is_label_index:
    print("'label' is part of the index.")
else:
    print("'label' is not part of the index.")

'label' is part of the index.


In [8]:
train.dtypes

pixel0      int64
pixel1      int64
pixel2      int64
pixel3      int64
pixel4      int64
            ...  
pixel779    int64
pixel780    int64
pixel781    int64
pixel782    int64
pixel783    int64
Length: 784, dtype: object

In [9]:
train.shape

(42000, 784)

## Preprocessing Data
>In order to prepare these images for a model I will change pixel values from int64 to float32. Each image is 28x28 which can be kept. The training and data sets will also be batched

In [10]:
train = train.astype("float32")
test = test.astype("float32")

In [11]:
#Normalize data
train = train / 255.0
test = test / 255.0

In [12]:
# Extracting the index as labels and resetting the index
y = train.index.values  # Extracting the index as labels
X = train.reset_index(drop=True).values  # Resetting the index and using remaining columns as features

In [13]:
# Split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape the training and validation data after splitting
X_train = X_train.reshape(-1, 28, 28, 1)
X_valid = X_valid.reshape(-1, 28, 28, 1)

In [14]:
test.reset_index(inplace=True)
test.columns

Index(['pixel0', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5', 'pixel6',
       'pixel7', 'pixel8', 'pixel9',
       ...
       'pixel774', 'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779',
       'pixel780', 'pixel781', 'pixel782', 'pixel783'],
      dtype='object', length=784)

In [15]:
# Convert the DataFrame to a NumPy array
test_array = test.values

# Reshape the array with the index as the first column
test_array = test_array.reshape(-1, 28, 28, 1)

In [16]:
test_array.shape

(28000, 28, 28, 1)

## Building a model

In [17]:
# Define a simple CNN architecture
# Example model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(300, activation="relu"),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

In [18]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
#Callback functions
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)


# Train the model
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2, callbacks=[early_stopping, 
                                                                                                 checkpoint,
                                                                                                 reduce_lr])

Epoch 1/5
840/840 [==============================] - 5s 5ms/step - loss: 0.2856 - accuracy: 0.9145 - val_loss: 0.1446 - val_accuracy: 0.9585 - lr: 0.0010
Epoch 2/5
 40/840 [>.............................] - ETA: 3s - loss: 0.1093 - accuracy: 0.9664

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


840/840 [==============================] - 4s 4ms/step - loss: 0.1113 - accuracy: 0.9667 - val_loss: 0.1213 - val_accuracy: 0.9659 - lr: 0.0010
Epoch 3/5
840/840 [==============================] - 4s 5ms/step - loss: 0.0694 - accuracy: 0.9782 - val_loss: 0.1158 - val_accuracy: 0.9667 - lr: 0.0010
Epoch 4/5
840/840 [==============================] - 4s 5ms/step - loss: 0.0522 - accuracy: 0.9828 - val_loss: 0.1289 - val_accuracy: 0.9613 - lr: 0.0010
Epoch 5/5
840/840 [==============================] - 4s 5ms/step - loss: 0.0372 - accuracy: 0.9879 - val_loss: 0.1149 - val_accuracy: 0.9702 - lr: 0.0010


In [24]:
# Predict on the test data
predictions = model.predict(test_array)

# Get the predicted digit for each image
predicted_digits = np.argmax(predictions, axis=1)  # Assuming predictions are one-hot encoded

# Create a DataFrame with ImageId and predicted digit
image_ids = np.arange(1, len(predicted_digits) + 1)  # ImageIds start from 1
submission_df = pd.DataFrame({'ImageId': image_ids, 'Label': predicted_digits})

# Save predictions to a CSV file in the required format
submission_df.to_csv('submission.csv', index=False)

875/875 [==============================] - 2s 2ms/step


## Results

>The submissions received an accuracy of 96.971%!! Not bad!